In [3]:
pip install langchain_exa


In [5]:
import os

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["EXA_API_KEY"] = "406dddf9-bbb0-4b88-8519-181b2d86adf1"

In [36]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = 'ls__3ab0148a8b414a50bf03f740d59373b2'
#os.environ["LANGCHAIN_PROJECT"] = ""

In [6]:
#import and initialize retriever
from langchain_exa import ExaSearchRetriever


retriever = ExaSearchRetriever(k=3, highlights=True)
documents = retriever.invoke("best time to visit japan")

In [14]:
print(documents[0].metadata['highlights'])
print(documents[0].metadata['url'])


[' Yes, it does rain in Japan, but you need it for those moody mists that make the temples look as if they are floating on air.   The best time to visit Japan is either spring (March to May) or autumn (September to November). Both seasons offer relatively good weather that\x92s ideal for walking. Spring is the right time to visit Japan to admire the cherry blossoms, while in the autumn the trees are resplendent with golds, reds and oranges. Japan is very much a four-season type of place to holiday, with fantastic festivals and events taking place throughout the year.']
https://www.responsibletravel.com/holidays/japan/travel-guide/best-time-to-visit-japan
{'title': 'Best time to visit Japan', 'url': 'https://www.responsibletravel.com/holidays/japan/travel-guide/best-time-to-visit-japan', 'id': 'VgapxBYpc_MMOnTbgkvdUg', 'score': 0.19090484082698822, 'published_date': '2019-10-08', 'author': 'Rob Perkins', 'highlights': [' Yes, it does rain in Japan, but you need it for those moody mists 

In [21]:
from pprint import pprint
pprint(documents[0].page_content)

(' \n'
 '  \n'
 '  When is the best time to visit Japan\n'
 '   \n'
 ' \n'
 ' \n'
 ' \n'
 ' Yes, it does rain in Japan, but you need it for those moody mists that make '
 'the temples look as if they are floating on air.\n'
 ' \n'
 ' \n'
 '  The best time to visit Japan is either spring (March to May) or autumn '
 '(September to November). Both seasons offer relatively good weather '
 'that\x92s ideal for walking. Spring is the right time to visit Japan to '
 'admire the cherry blossoms, while in the autumn the trees are resplendent '
 'with golds, reds and oranges. Japan is very much a four-season type of place '
 'to holiday, with fantastic festivals and events taking place throughout the '
 'year. The climate varies from the cooler, mountainous north to the '
 'subtropical islands of the south with their Pacific vibe.\n'
 ' \n'
 ' \n'
 ' When is the best time to visit Japan for the cherry blossoms?  Mid-March to '
 'April is the best time to see the cherry blossoms in Japan, as they

In [31]:
from langchain_exa import ExaSearchRetriever
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda


#helps to separate the inputed documents a litle better when html
document_prompt = PromptTemplate.from_template(""" 
<source>
    <url>{url}</url>
    <highlights>{highlights}</highlights>
</source>
    """)
retriever = ExaSearchRetriever(k=3, highlights=True)

document_chain = RunnableLambda(lambda document: 
    {'highlights': document.metadata['highlights'], 
     'url' : document.metadata['url'] 
    }
) | document_prompt

#map() is because retriever spits k=3 documents. We apply document chain for each of them 
retrieval_chain = retriever | document_chain.map()  | (lambda docs : "/n".join([i.text for i in docs]))

pprint(retrieval_chain.invoke("What is the capital of France?"))

print('step')

(' \n'
 '<source>\n'
 '    <url>https://worldpopulationreview.com/countries/france/capital</url>\n'
 "    <highlights>['Paris is located at 48.8566, 2.3522, at an elevation of "
 '115 ft.   Eiffel Tower, Paris, France  Source: '
 'https://flic.kr/p/HbwPjZ       2023 Population 11,208,440   2023 Growth Rate '
 '0.59%   Founded 3rd century   Capital as of 1944   Elevation 115   Largest '
 'City Yes   Role in Country Houses the political and economical '
 'Luxembour        What is the Capital of France? Paris Shown on a Map     '
 "']</highlights>\n"
 '</source>\n'
 '    /n \n'
 '<source>\n'
 '    <url>https://simple.wikipedia.org/wiki/Capital_of_France</url>\n'
 "    <highlights>['Tours (1589—1594) faction of parliamentarians, faithful to "
 'the king was sitting at Tours. Paris (1661-1682) Paris is the official '
 'capital, but Louis XIV disputed that Versalles will be the politic centre.  '
 'Versailles (1682–1715) In 1682, Louis XIV made Versailles his residence. '
 'Versailles (1722–

In [34]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model= 'gpt-3.5-turbo', temperature = 0)

generation_prompt = ChatPromptTemplate.from_messages({
    ('system', "You are an expert research assistant. You use xml-formated context to research people questions"),
    ('human' , 
    """
Please answer the following query based on the provided context. Please cite your sources at the end of your response.:
     
Query: {query}
---
<context>
{context}
</context>""")
})

chain = RunnableParallel({
    "query" : RunnablePassthrough(),
    "context" : retrieval_chain
}) | generation_prompt | llm

In [38]:
response = chain.invoke("Best time to visit Alps")



In [39]:
print(response.content)

Based on the provided context, the best time to visit the Alps is during the summer months. More and more people are choosing to visit the Alps in the summer to enjoy the mountain air, be active in nature, and avoid the crowds of the peak winter season. The Alps offer fantastic hiking paths, stunning views of lakes and valleys, and a variety of outdoor activities during the summer months.

Sources:
1. National Geographic - "Choose the Alps for Your Next Summer Adventure": [Link](https://www.nationalgeographic.com/travel/article/choose-the-alps-for-your-next-summer-adventure)
2. Conde Nast Traveller - "Best Places to Go Skiing in Switzerland": [Link](https://www.cntraveller.com/gallery/best-places-to-go-skiing-switzerland?utm_brand=conde-nast-traveller-uk&utm_medium=social&utm_source=twitter&utm_social-type=owned)
3. Austria.info - "Tirol Province": [Link](https://www.austria.info/en/where-to-go/provinces/tirol)


In [40]:
pip install --upgrade langchain cli



     -------------------------------------- 807.5/807.5 kB 4.3 MB/s eta 0:00:00


ERROR: Could not find a version that satisfies the requirement cli (from versions: none)
ERROR: No matching distribution found for cli
